<a href="https://colab.research.google.com/github/krissivolga29-commits/analytics_big_data/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import bz2
import gdown
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import re
import regex

from collections import Counter
from contextlib import redirect_stdout
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from string import punctuation
from tqdm import tqdm

from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

from sklearn.manifold import TSNE
from sklearn.metrics  import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import LabelPropagation, LabelSpreading

In [ ]:
!pip3 install gensim

In [ ]:
from gensim import similarities
from gensim.corpora import Dictionary
from gensim.models import KeyedVectors, lsimodel, TfidfModel, word2vec

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
! pip install igraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.4 MB/s eta 0:00:00


In [ ]:
import igraph as ig

In [ ]:
!pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 57.4 MB/s eta 0:00:00


In [ ]:
import pymorphy3
from pymorphy3 import MorphAnalyzer
pymorphy3_analyzer = MorphAnalyzer()

### Необходимые функции

In [ ]:
mystopwords = stopwords.words('russian')

ru_words = re.compile("[А-Яа-я]+")

def words_only(text):
    return " ".join(ru_words.findall(text))

def lemmatize(text):
    try:
        return  " ".join([morph.parse(word)[0].normal_form for word in text.lower().split()])
    except:
        return " "

def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

def preprocess(text):
    return remove_stopwords(lemmatize(words_only(text.lower())))

### Скачивание данных

In [ ]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:33, 2150.79it/s]


In [ ]:
len(responses)

153499

In [ ]:
df = pd.DataFrame(responses)
df.head()

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,г. Москва,False,Жалоба,0,лицензия № 2562,uhnov1,Бинбанк,2015-06-08 12:50:54,Добрый день! Я не являюсь клиентом банка и пор...,NaN
1,г. Новосибирск,False,Не могу пользоваться услугой Сбербанк он-лайн,0,лицензия № 1481,Foryou,Сбербанк России,2015-06-08 11:09:57,Доброго дня! Являюсь держателем зарплатной кар...,NaN
2,г. Москва,False,Двойное списание за один товар.,1,лицензия № 2562,Vladimir84,Бинбанк,2015-06-05 20:14:28,Здравствуйте! Дублирую свое заявление от 03.0...,NaN
3,г. Ставрополь,False,Меняют проценты комиссии не предупредив и не ...,2,лицензия № 1481,643609,Сбербанк России,2015-06-05 13:51:01,Добрый день!! Я открыл расчетный счет в СберБа...,NaN
4,г. Челябинск,False,Верните денежные средства за страховку,1,лицензия № 2766,anfisa-2003,ОТП Банк,2015-06-05 10:58:12,"04.03.2015 г. взяла кредит в вашем банке, заяв...",NaN


###**Задание 1:** составить словари для классификации по тональности.
Данные для задания — отзывы
на банки, собранные с нескольких сайтов рунета. Отзывы могут быть как положительными

— оценка 5, так и отрицательными — оценка 1.

1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова.
Обучите по коллекции предложений word2vec

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
sentences = []
for i in range(len(df)):
    sentences.extend(sent_tokenize(df.text.iloc[i]))

In [ ]:
sentences[:5]

['Добрый день!',
 'Я не являюсь клиентом банка и поручителем по кредитному договору, а также не являюсь каким-либо другим лицом, письменно  оформившим отношения с банком по поводу урегулирования чьей-либо  задолженности.',
 'Начиная с марта 2015 года начали приходить бесконечные письма из ООО "Примо коллект"на мой адрес: город Москва, Уваровский переулок, дом 10, квартира 111, с угрозами о возбуждении уголовного дела в отношении гражданина Филиппова Эдуарда Владимировича, который уклоняется от уплаты взятых им кредитов: договор № 81014 от 20.10.2013 года и договор № 2464946 от 09.10.2014 года.',
 'Со всей ответственностью\xa0 хочу Вас заверить, что вышеуказанный гражданин, которого Вы разыскиваете, мне не знаком и никогда в моем адресе не был зарегистрирован.',
 'Каким образом Вы не удостоверившись в подлинности его документов оформили на его имя кредитный договор, мне по меньшей мере не понятно,\xa0 и почему по Вашей милости я должна переживать и бояться за себе и свое имущество.']

In [ ]:
morph = pymorphy3.MorphAnalyzer()

In [ ]:
lemmas = []
for sentence in tqdm(sentences):
    lemmas.append(preprocess(sentence))

100%|██████████| 2664911/2664911 [1:33:38<00:00, 474.30it/s]


In [ ]:
lemmas[:5]

['добрый день',
 'являться клиент банк поручитель кредитный договор также являться либо лицо письменно оформить отношение банк повод урегулирование чей либо задолженность',
 'начинать март год начать приходить бесконечный письмо ооо прить коллект адрес город москва уваровский переулок дом квартира угроза возбуждение уголовный дело отношение гражданин филиппов эдуард владимирович который уклоняться уплата взять кредит договор год договор год',
 'весь ответственность хотеть заверить вышеуказанный гражданин который разыскивать знак адрес зарегистрировать',
 'образ удостовериться подлинность документ оформить имя кредитный договор малый мера понятно почему ваш милость должный переживать бояться свой имущество']

### Обучение по коллекции предложений word2vec.

In [ ]:
words = [sentence.split() for sentence in lemmas]

In [ ]:
model_response = word2vec.Word2Vec(words, workers=4, vector_size=200, min_count=3, window=5, epochs=15)

In [ ]:
model_response.save("model_w2v.model")

2. Приведите несколько удачных и неудачных примеров решения стандартных
текстов для word2vec:

- тест на определение ближайших слов

- тест на аналогии (мужчина — король : женщина — королева)

- тест на определение лишнего слова

In [ ]:
# ближайшие слова
model_response.wv.most_similar('зарплата')

[('зп', 0.8929630517959595),
 ('стипендия', 0.7276934385299683),
 ('пенсия', 0.6819296479225159),
 ('заработный', 0.6593749523162842),
 ('гонорар', 0.6589528918266296),
 ('з', 0.6110963821411133),
 ('пособие', 0.5967647433280945),
 ('доход', 0.5744452476501465),
 ('оклад', 0.5574368238449097),
 ('зара', 0.5573235750198364)]

In [ ]:
# аналоги
model_response.wv.most_similar(positive=['женщина', 'человек'], negative=['мужчина'])

[('жь', 0.6075681447982788),
 ('особа', 0.567215085029602),
 ('девчонка', 0.5492603182792664),
 ('парень', 0.5110949873924255),
 ('дама', 0.5062927603721619),
 ('дарование', 0.4976176917552948),
 ('девочка', 0.4946194887161255),
 ('учёный', 0.4918697774410248),
 ('мамочка', 0.48130419850349426),
 ('бабушка', 0.4781856834888458)]

In [ ]:
# определение лишнего слова
model_response.wv.doesnt_match('вклад прибыль доход рост кредит'.split())

'кредит'

**Вывод:** тесты показали, что модель хорошо обучилась, хотя мы видим, странное слово 'жь' во втором тесте.

3. Постройте несколько визуализаций:

- TSNE для топ-100 или топ-500 слов и найдите осмысленные кластеры слов
- задайте координаты для нового пространства следующим образом: одна ось описывает отношение «плохо — хорошо», вторая — «медленно — быстро», и найдите координаты названий банков в этих координатах

### Построение визуализаций:

TSNE для топ-100 слов и найти осмысленные кластеры слов.

In [ ]:
top_words = []
fd = FreqDist()
for word in words:
    fd.update(word)
for idx in fd.most_common(100):
    top_words.append(idx[0])
print(top_words)

['банк', 'карта', 'это', 'деньга', 'день', 'всё', 'кредит', 'который', 'отделение', 'клиент', 'сотрудник', 'мочь', 'счёт', 'свой', 'сказать', 'сумма', 'заявление', 'год', 'вопрос', 'время', 'ещё', 'получить', 'кредитный', 'весь', 'ответ', 'вклад', 'офис', 'телефон', 'платёж', 'договор', 'ваш', 'дать', 'номер', 'звонить', 'банкомат', 'сбербанк', 'очень', 'позвонить', 'г', 'месяц', 'написать', 'проблема', 'работать', 'средство', 'рубль', 'работа', 'документ', 'ситуация', 'звонок', 'прийти', 'решить', 'говорить', 'человек', 'хотеть', 'операция', 'почему', 'должный', 'информация', 'сделать', 'смс', 'срок', 'услуга', 'заявка', 'знать', 'просто', 'обслуживание', 'очередь', 'минута', 'обратиться', 'данный', 'большой', 'первый', 'сегодня', 'девушка', 'ждать', 'сайт', 'руб', 'претензия', 'новый', 'просить', 'интернет', 'процент', 'неделя', 'никакой', 'оператор', 'сообщить', 'ответить', 'являться', 'далее', 'стать', 'нужно', 'час', 'несколько', 'оформить', 'перевод', 'писать', 'комиссия', 'следу

In [ ]:
top_words_vec = model_response.wv[top_words]
tsne = TSNE(n_components=2, random_state=21)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [ ]:
output_notebook()

p = figure(tools = 'pan, wheel_zoom, reset, save',
           toolbar_location = 'above',
           title = 'TSNE для топ-100 слов')

source = ColumnDataSource(data = dict(x=top_words_tsne[:,0],
                                      y=top_words_tsne[:,1],
                                      names=top_words))

p.scatter(x='x', y='y', size=6, source=source)

labels = LabelSet(x='x', y='y', text = 'names', y_offset = 6,
                  text_font_size = '8pt', text_color = 'blue',
                  source = source, text_align = 'center')
p.add_layout(labels)

show(p)

### координаты для нового пространства

In [ ]:
bank_list = ['сбербанк','альфа','втб','бинбанк', 'хоум', 'тинькофф',
             'ренессанс', 'отп', 'ситибанк', 'авангард',
             'промсвязьбанк', 'юникредит', 'мтс', 'росбанк', 'уралсиб']
bank_list_vec = model_response.wv[bank_list]

In [ ]:
x_axis = model_response.wv['хороший'] - model_response.wv['плохой']
y_axis =  model_response.wv['быстрый'] - model_response.wv['медленный']

bank_list_x = []
bank_list_y = []
for el in bank_list_vec:
    bank_list_x.append(np.dot(el, x_axis))
    bank_list_y.append(np.dot(el, y_axis))

p = figure(tools="pan, wheel_zoom, reset, save",
           toolbar_location="above",
           x_axis_label = "плохой --> хороший",
           y_axis_label = "медленный --> быстрый",
           title="TSNE для банков")

source = ColumnDataSource(data=dict(x=bank_list_x,
                                    y=bank_list_y,
                                    names=bank_list))

p.scatter(x="x", y="y", size=8, source=source)

labels = LabelSet(x="x", y="y", text="names", y_offset=6,
                  text_font_size="8pt", text_color="blue",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

###**Задание 2:** распространить метку.

1. Определите 5–8 позитивных слов (например, быстрый, удобный) и 5–8 негативных слов (например, очередь, медленно). Эти слова будут основной будущего оценочного словаря

Будем считать, что положительному классу соответствует 1, а негативному соответствует 0

In [ ]:
positive = ['быстрый', 'хороший', 'легко', 'комфортно',
            'отлично', 'нравиться','удобный', 'отличный']
negative = ['долго', 'медленный', 'плохой', 'ужасно',
            'отвратительный', 'тяжело', 'скверный', 'безобразно']

2. Пусть позитивному классу соответствует метка 1, негативному — -1

In [ ]:
g = ig.Graph(directed=True)
for word in model_response.wv.key_to_index.keys():
    g.add_vertex(word)

for word in tqdm(model_response.wv.key_to_index.keys()):
    node = g.vs.select(name = word).indices[0]
    similar_words = model_response.wv.most_similar(word, topn=5)
    for sim in similar_words:
        word1 = sim[0]
        val  = sim[1]
        new_node = g.vs.select(name = word1).indices[0]
        g.add_edge(node, new_node, weight = val)

100%|██████████| 42565/42565 [1:21:37<00:00,  8.69it/s]


In [ ]:
ig.summary(g)

IGRAPH DNW- 42565 212825 -- 
+ attr: name (v), weight (e)


Обход ограничения на размер вывода при использовании графа.

In [ ]:
with open('out.txt', 'w') as f:
    with redirect_stdout(f):
        print(g)

In [ ]:
with open('out.txt') as f:
  l = [line.strip() for line in f]

In [ ]:
l[:20]

['IGRAPH DNW- 42565 212825 --',
 '+ attr: name (v), weight (e)',
 '+ edges (vertex names):',
 'банк -> сбербанк, ситибанк, росбанк, промсвязьбанк, брс',
 'карта -> карточка, дебетовый, кредитка, кк, дебетовка',
 'это -> данный, хотя, подобный, умыслить, невосполнять',
 'деньга -> средство, денежка, наличка, дс, денюжка',
 'день -> месяц, неделя, час, сутки, понедельник',
 'всё -> вс, ладный, вроде, уса, воспрепятствование',
 'кредит -> ипотека, автокредит, потреба, потребкредить,',
 'ссуда',
 'который -> хотя, поскольку, соответственно, следовательно,',
 'посторонний',
 'отделение -> офис, филиал, осб, допофис, отд',
 'клиент -> вкладчик, пользователь, мщиковы, клиентура,',
 'клиенто',
 'сотрудник -> специалист, менеджер, сотрудница, представитель,',
 'работник',
 'мочь -> хотеть, нужно, смочь, возможно, невозможно',
 'счёт -> сч, сберкнижка, картсчёт, скс, картсча']

3. Пометьте выбранные слова в лексическом графе соответствующими метками

In [ ]:
def marking():
    '''Маркировка'''
    for word in tqdm(words):
        label = int(spread_df['label'][spread_df['word'] == word])
        if label != -1:
            get_label([word], label)

def get_label(word, label):
    '''
    Ищит близкие слова. Если близость больше 0.6, то выставляет матку, как у первоначального слова.
    '''
    seq_s = model_response.wv.most_similar(word, topn=10)
    for seq in seq_s:
        if seq[1] > 0.6:
            spread_df['label'][spread_df['word'] == seq[0]] = label

In [ ]:
words = model_response.wv.index_to_key
vectors = [model_response.wv[i] for i in words]
labels = []
for w in words:
  if w in positive: labels.append(1)
  elif w in negative: labels.append(0)
  else: labels.append(-1)

In [ ]:
spread_df = pd.DataFrame(zip(words, labels), columns=['word', 'label'])

In [ ]:
marking()

100%|██████████| 42565/42565 [05:21<00:00, 132.44it/s]


In [ ]:
spread_df[spread_df['label'] != -1]

,word,label
36,очень,0
127,хороший,1
310,долго,0
459,удобный,1
524,довольный,1
...,...,...
42473,скрючить,1
42474,котляковский,1
42492,швец,1
42536,берсенев,1


In [ ]:
marking()

100%|██████████| 42565/42565 [08:54<00:00, 79.70it/s]


In [ ]:
spread_df[spread_df['label'] != -1]

,word,label
33,звонить,1
36,очень,0
114,приходить,1
127,хороший,1
163,объяснить,1
...,...,...
42517,плого,1
42526,кольчугино,1
42536,берсенев,1
42552,вжжик,1


In [ ]:
label_model = LabelSpreading(kernel='knn', alpha=0.2, n_neighbors=7, n_jobs=4, max_iter=400)

In [ ]:
model_df = pd.concat([spread_df, pd.DataFrame(vectors)], axis=1)

In [ ]:
model_df = model_df[model_df['label'] != -1]
y = model_df['label']
X = model_df.drop(columns=['word', 'label'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [ ]:
label_model.fit(X_train, y_train)

LabelSpreading(kernel='knn', max_iter=400, n_jobs=4)

Произведём маркировку лексического словаря на основе обученной модели LabelSpreading и проверим её точность.

In [ ]:
spread_df['prediction'] = spread_df.word.apply(lambda x: label_model.predict([model_response.wv[x]])[0])

In [ ]:
predict = label_model.predict(X_test)
report = classification_report(y_test, predict)
print(report)

              precision    recall  f1-score   support

           0       0.79      0.19      0.30        59
           1       0.94      1.00      0.97       764

    accuracy                           0.94       823
   macro avg       0.86      0.59      0.63       823
weighted avg       0.93      0.94      0.92       823



In [ ]:
result = spread_df[spread_df['label'] == -1]
result.head(20)

,word,label,prediction
0,банк,-1,1
1,карта,-1,1
2,это,-1,1
3,деньга,-1,1
4,день,-1,1
5,всё,-1,1
6,кредит,-1,1
7,который,-1,1
8,отделение,-1,1
9,клиент,-1,1



**Выводы:**

В рамках работы был составлен словарь для последующей классификации по тональности. На основе словаря были построены визуализации с применением TSNE.

Также была произведено распространение метки в лексическом словаре: до применение модели LabelPropagation была произведена разметка словаря по ближайшим словам к размеченным. Точность модели была получена равная 0.94